In [2]:
%pip uninstall -y torch torchvision torchdata torchaudio
%pip install -U torch==1.13.1 torchdata

%pip uninstall -y ray transformers datasets
%pip install -U transformers datasets ray==2.2.0

Found existing installation: torch 1.13.1
Uninstalling torch-1.13.1:
  Successfully uninstalled torch-1.13.1
Found existing installation: torchdata 0.5.1
Uninstalling torchdata-0.5.1:
  Successfully uninstalled torchdata-0.5.1
Note: you may need to restart the kernel to use updated packages.
  Using cached torch-1.13.1-cp38-cp38-manylinux1_x86_64.whl (887.4 MB)
  Using cached torchdata-0.5.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Found existing installation: ray 2.2.0
Uninstalling ray-2.2.0:
  Successfully uninstalled ray-2.2.0
Found existing installation: transformers 4.26.1
Uninstalling transformers-4.26.1:
  Successfully uninstalled transformers-4.26.1
Found existing installation: datasets 2.9.0
Uninstalling datasets-2.9.0:
  Successfully uninstalled datasets-2.9.0
Note: you may need t

In [3]:
!pip list | grep torch

sagemaker-pytorch-training 2.7.0
torch                      1.13.1
torchdata                  0.5.1

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [4]:
# restart kernel to pick up the pip installs above
import IPython

IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel


{'status': 'ok', 'restart': True}

In [5]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = ""

#os.environ["PL_TORCH_DISTRIBUTED_BACKEND"] = "gloo"

In [6]:
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer

#model_checkpoint = "gpt2"

model_checkpoint = "bigscience/bloom-560m"

#tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"
block_size = 128

#datasets = load_dataset("wikitext", "wikitext-2-raw-v1")

dataset_train = load_dataset("amazon_us_reviews", "Digital_Video_Games_v1_00") #, data_files={"train": "Apparel_v1_00"})  #, "validation": path_to_validation.txt}

dataset_validation = load_dataset("amazon_us_reviews", "Digital_Software_v1_00") #, data_files={"train": "Apparel_v1_00"})  #, "validation": path_to_validation.txt}

# data_files = {}
# data_files["train"] = f"{data_path}/test/part-algo-1-womens_clothing_ecommerce_reviews.csv"
# extension = "csv"

# raw_datasets = load_dataset(extension, data_files=data_files)


tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

text_column_name = "review_body"

def tokenize_function(examples):
    tokenized = tokenizer(examples[text_column_name])
    return tokenized
    
tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True, num_proc=4, remove_columns=[
    'marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 
    'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 
    'review_headline', 'review_date', text_column_name])

tokenized_dataset_validation = dataset_validation.map(tokenize_function, batched=True, num_proc=4, remove_columns=[
    'marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 
    'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 
    'review_headline', 'review_date', text_column_name])

print(tokenized_dataset_train["train"][1])
print(tokenized_dataset_validation["train"][1])

block_size = 128

# tokenized_datasets = datasets.map(
#     tokenize_function, batched=True, num_proc=1, remove_columns=["text"]
# )


def group_texts(examples):    
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


lm_dataset_train = tokenized_dataset_train.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=1,
)

lm_dataset_validation = tokenized_dataset_validation.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=1,
)


/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset amazon_us_reviews (/root/.cache/huggingface/datasets/amazon_us_reviews/Digital_Video_Games_v1_00/0.1.0/17b2481be59723469538adeb8fd0a68b0ba363bbbdd71090e72c325ee6c7e563)
100%|██████████| 1/1 [00:00<00:00, 189.52it/s]
Found cached dataset amazon_us_reviews (/root/.cache/huggingface/datasets/amazon_us_reviews/Digital_Software_v1_00/0.1.0/17b2481be59723469538adeb8fd0a68b0ba363bbbdd71090e72c325ee6c7e563)
100%|██████████| 1/1 [00:00<00:00, 232.75it/s]
Loading cached processed dataset at /root/.cache/huggingface/datasets/amazon_us_reviews/Digital_Video_Games_v1_00/0.1.0/17b2481be59723469538adeb8fd0a68b0ba363bbbdd71090e72c325ee6c7e563/cache-f73925383eaf671a.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/a

{'input_ids': [121605, 80578], 'attention_mask': [1, 1]}
{'input_ids': [19405, 25689, 267, 10512, 3172, 2909, 37663], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


In [ ]:
import ray
ray.shutdown()

from ray.train.huggingface import HuggingFaceTrainer
from ray.air.config import ScalingConfig, RunConfig
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from ray.tune import SyncConfig


# ray.init(runtime_env={"pip": [
#                         "torch", 
#                         "scikit-learn",
#                         "transformers",
#                         "pandas",
#                         "datasets",
#                         "accelerate",
#                         "scikit-learn",
#                         "mlflow", 
#                         "tensorboard",
#                         "s3fs"]
#                      }
#         )

def trainer_init_per_worker(train_dataset, 
                            eval_dataset, 
                            **config):
    #model_config = AutoConfig.from_pretrained(model_checkpoint)
    #model = AutoModelForCausalLM.from_config(model_config)

    model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
    
    args = TrainingArguments(
        output_dir=f"{model_checkpoint}-finetuned-amazon-customer-reviews",
#        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=2e-5,
        weight_decay=0.01,  
        max_steps=10,
#        eval_steps=10,
        num_train_epochs=1.0,
        no_cuda=True        
    )
    
    return Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
#        eval_dataset=eval_dataset,
    )


ray_train_ds = ray.data.from_huggingface(
    lm_dataset_train["train"]
)

ray_evaluation_ds = ray.data.from_huggingface(
    lm_dataset_validation["train"]
)


s3_checkpoint_prefix="s3://dsoaws/ray_output"

import multiprocessing
num_cpus = multiprocessing.cpu_count()

scaling_config = ScalingConfig(num_workers=1, 
                               #use_gpu=True,
                               trainer_resources={"CPU": num_cpus - 2}, #, "GPU": 0},
                               #_max_cpu_fraction_per_node = 0.8
                              )

# If using GPUs, use the below scaling config instead.
# scaling_config = ScalingConfig(num_workers=3, use_gpu=True)
trainer = HuggingFaceTrainer(
    trainer_init_per_worker=trainer_init_per_worker,
    scaling_config=scaling_config,
    run_config = RunConfig(
        sync_config=SyncConfig(
            # This will store checkpoints in S3.
            upload_dir=s3_checkpoint_prefix
        )
    ),
    datasets={"train": ray_train_ds, 
              #"evaluation": ray_evaluation_ds
             },
)
result = trainer.fit()

2023-02-15 21:02:20,906	INFO worker.py:1538 -- Started a local Ray instance.
2023-02-15 21:02:22,648	INFO data_parallel_trainer.py:286 -- GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.
/opt/conda/lib/python3.8/site-packages/ray/train/base_trainer.py:354: UserWarning: Executing `.fit()` may leave less than 20% of CPUs in this cluster for Dataset execution, which can lead to resource contention or hangs. To avoid this, reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer scaling_config. See https://docs.ray.io/en/master/data/dataset-internals.html#datasets-and-tune for more info.
  tuner = Tuner(trainable=trainable, run_config=self.run_config)
2023-02-15 21:02:22,727	INFO tensorboardx.py:170 -- pip install "ray[tune]" to see TensorBoard files.
2023-02-15 21:02:22,728	WARNING callback.py:108 -- The Tens

(HuggingFaceTrainer pid=14576) 2023-02-15 21:02:25,728	INFO data_parallel_trainer.py:286 -- GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.
(RayTrainWorker pid=14712) 2023-02-15 21:02:27,725	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=14712) max_steps is given, it will override any value given in num_train_epochs
(RayTrainWorker pid=14712) /opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(RayTrainWorker pid=14712)   warnings.warn(
(RayTrainWorker pid=14712) ***** Running training *****
(RayTrainWorker pid=14712)   Num examples = 92921
(RayTrainWorker pid=14712)   Num Epo

(RayTrainWorker pid=14712) {'loss': 4.0161, 'learning_rate': 0.0, 'epoch': 0.0}


100%|██████████| 10/10 [25:30<00:00, 152.52s/it]Saving model checkpoint to bigscience/bloom-560m-finetuned-amazon-customer-reviews/checkpoint-10
(RayTrainWorker pid=14712) Configuration saved in bigscience/bloom-560m-finetuned-amazon-customer-reviews/checkpoint-10/config.json
(RayTrainWorker pid=14712) Configuration saved in bigscience/bloom-560m-finetuned-amazon-customer-reviews/checkpoint-10/generation_config.json
(RayTrainWorker pid=14712) Model weights saved in bigscience/bloom-560m-finetuned-amazon-customer-reviews/checkpoint-10/pytorch_model.bin


(RayTrainWorker pid=14712) {'train_runtime': 1588.9661, 'train_samples_per_second': 0.05, 'train_steps_per_second': 0.006, 'train_loss': 4.016141891479492, 'epoch': 0.0}


(RayTrainWorker pid=14712) 
(RayTrainWorker pid=14712) 
(RayTrainWorker pid=14712) Training completed. Do not forget to share your model on huggingface.co/models =)
(RayTrainWorker pid=14712) 
(RayTrainWorker pid=14712) 
100%|██████████| 10/10 [26:28<00:00, 158.90s/it]


Trial name,_time_this_iter_s,_timestamp,_training_iteration,date,done,episodes_total,epoch,experiment_id,hostname,iterations_since_restore,learning_rate,loss,node_ip,pid,should_checkpoint,step,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_runtime,train_samples_per_second,train_steps_per_second,training_iteration,trial_id,warmup_time
HuggingFaceTrainer_0acb7_00000,1595.64,1676496544,1,2023-02-15_21-29-04,False,,0.000860882,c58abde027b646adb64519e3aa670960,pytorch-1-12-cpu-py3-ml-g5-8xlarge-dadc2efa7892c7009794b397aac6,1,0,4.0161,169.255.254.2,14576,True,10,1598.64,1598.64,1598.64,1676496544,0,,4.01614,1588.97,0.05,0.006,1,0acb7_00000,0.00494528


In [ ]:
result.checkpoint

In [ ]:
TODO.save_pretrained('s3://dsoaws/bloom/output/model_from_notebook_ray/')

In [ ]:
import transformers
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bigscience/bloom-560m', use_fast=True)
#model = AutoModelForCausalLM.from_pretrained('s3://dsoaws/bloom/output/model_from_notebook_ray/')
                                             

In [ ]:
type(trainer.trainer_init_per_worker.model)

In [ ]:
prompt = "Norton Antivirus"
result_length = 100
inputs = tokenizer(prompt, return_tensors="pt")

print(inputs)


In [ ]:
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=result_length, 
                       do_sample=True, 
                       top_k=50, 
                       top_p=0.9
                      )[0]))